In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [2]:
import os
from torch_geometric.datasets import TUDataset
import os.path as osp
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseSAGEConv, dense_diff_pool,dense_mincut_pool
import torch_geometric.transforms as T

import sys
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/")
from sklearn.cluster import KMeans
import networkx as nx
import numpy as np
from graph_generation.load_graphs_and_create_metadata import dataset_metadata
from graph_matching_tools.metrics import matching
import matplotlib.pyplot as plt
import scipy.io as sco
import slam.io as sio
from scipy.special import softmax
import pickle
from scipy.stats import betabinom
import seaborn as sns
import tools.graph_processing as gp
import tools.graph_visu as gv
from matplotlib.pyplot import figure
import pandas as pd
import random
from torch_geometric.utils.convert import from_networkx
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch.nn.functional import one_hot
from sklearn.preprocessing import OneHotEncoder
from torch.nn import Linear
import torch.nn.functional as F
from math import ceil
import torch_geometric as pyg
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import TopKPooling
from torch_geometric.data import Data
from torch_geometric.loader import DenseDataLoader
from torch.nn.utils.rnn import pad_sequence
from torch_geometric.utils import to_dense_batch, to_dense_adj
from torch_geometric.nn import GCNConv, DenseGraphConv
%matplotlib inline

In [3]:
path_trials = '..//data/simu_graph/simu_genders//'
path_sub_dir = '/noise_100,outliers_varied/graphs/'
all_trials = np.sort(list(map(int,os.listdir(path_trials)))) # all trials

In [4]:
all_graphs_same_trial = []
all_graphs_labels_same_trial = []

referene_graphs = []

for trial in all_trials:
    
    #referene_graphs.append(nx.read_gpickle(path_trials + str(trial) + '/reference_'+ str(trial) +'.gpickle'))
    
    trial_folder = path_trials + str(trial) + path_sub_dir
    all_graphs_same_trial.append(gp.load_graphs_in_list(trial_folder)) # append graphs
    all_graphs_labels_same_trial.append(np.ones([len(gp.load_graphs_in_list(trial_folder))])*trial) # append corr lab

In [5]:
# taking 300 graph from each population for learning 

simu_train_set = []
simu_train_labels = []

for graph_set,graph_labels in zip(all_graphs_same_trial,all_graphs_labels_same_trial):
    simu_train_set.extend(graph_set[:300])
    simu_train_labels.extend(graph_labels[:300])

In [6]:
def graph_remove_dummy_nodes(graph):
    nodes_dummy_true = [x for x,y in graph.nodes(data=True) if y['is_dummy']==True]
    graph.remove_nodes_from(nodes_dummy_true)
    #print(len(graph.nodes))

In [7]:
# Convert networkx graphs to pyg graphs

sulcal_simu_dataset = []

for i,g in enumerate(simu_train_set):
    #graph_remove_dummy_nodes(g) # remove dummy nodes
    g.remove_edges_from(nx.selfloop_edges(g)) # remove self loop edges
    
    attr_coords = np.array(list(nx.get_node_attributes(g,'coord').values())) #simu attribute (coords)
    
    x = torch.tensor(attr_coords,dtype=torch.float)
    
    #adj = torch.tensor(nx.adjacency_matrix(g).todense(),dtype=torch.float)
    
    y = torch.tensor(simu_train_labels[i],dtype=torch.long) # add graph label
    edge_index = torch.tensor(list(g.edges))
    
    sulcal_simu_dataset.append(Data(x=x, y=y, edge_index=edge_index.t().contiguous())) #

/tmp/ipykernel_7111/1758284362.py:15: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  y = torch.tensor(simu_train_labels[i],dtype=torch.long) # add graph label


In [8]:
len(sulcal_simu_dataset)

3000

In [9]:
random.shuffle(sulcal_simu_dataset)

train_dataset = sulcal_simu_dataset[:len(sulcal_simu_dataset)-500]
test_dataset = sulcal_simu_dataset[len(sulcal_simu_dataset)-500:]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # dense data loader for using adjacency
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [10]:
num_node_features = train_dataset[0].num_features
num_node_features

3

In [11]:
# max_nodes = 150
# class MyFilter(object):
#     def __call__(self, data):
#         return data.num_nodes <= max_nodes


# path = osp.join(osp.dirname(osp.realpath('./tmp')), '..', 'data',
#                 'PROTEINS_dense')
# dataset = TUDataset(path, name='PROTEINS', transform=T.ToDense(max_nodes),
#                     pre_filter=MyFilter())
# dataset = dataset.shuffle()
# n = (len(dataset) + 9) // 10
# test_dataset = dataset[:n]
# val_dataset = dataset[n:2 * n]
# train_dataset = dataset[2 * n:]
# test_loader = DenseDataLoader(test_dataset, batch_size=20)
# val_loader = DenseDataLoader(val_dataset, batch_size=20)
# train_loader = DenseDataLoader(train_dataset, batch_size=20)

In [12]:
# class GNN(torch.nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels,
#                  normalize=False, lin=True):
#         super().__init__()

#         self.conv1 = DenseSAGEConv(in_channels, hidden_channels, normalize)
#         self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
#         self.conv2 = DenseSAGEConv(hidden_channels, hidden_channels, normalize)
#         self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
#         self.conv3 = DenseSAGEConv(hidden_channels, out_channels, normalize)
#         self.bn3 = torch.nn.BatchNorm1d(out_channels)

#         if lin is True:
#             self.lin = torch.nn.Linear(2 * hidden_channels + out_channels,
#                                        out_channels)
#         else:
#             self.lin = None

#     def bn(self, i, x):
#         batch_size, num_nodes, num_channels = x.size()

#         x = x.view(-1, num_channels)
#         x = getattr(self, f'bn{i}')(x)
#         x = x.view(batch_size, num_nodes, num_channels)
#         return x

#     def forward(self, x, adj, mask=None):
#         batch_size, num_nodes, in_channels = x.size()

#         x0 = x
#         x1 = self.bn(1, self.conv1(x0, adj, mask).relu())
#         x2 = self.bn(2, self.conv2(x1, adj, mask).relu())
#         x3 = self.bn(3, self.conv3(x2, adj, mask).relu())

#         x = torch.cat([x1, x2, x3], dim=-1)

#         if self.lin is not None:
#             x = self.lin(x).relu()

#         return x


# class Net(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         num_nodes = ceil(0.25 * max_nodes)
#         self.gnn1_pool = GNN(dataset.num_features, 64, num_nodes)
#         self.gnn1_embed = GNN(dataset.num_features, 64, 64, lin=False)

#         num_nodes = ceil(0.25 * num_nodes)
#         self.gnn2_pool = GNN(3 * 64, 64, num_nodes)
#         self.gnn2_embed = GNN(3 * 64, 64, 64, lin=False)

#         self.gnn3_embed = GNN(3 * 64, 64, 64, lin=False)

#         self.lin1 = torch.nn.Linear(3 * 64, 64)
#         self.lin2 = torch.nn.Linear(64, dataset.num_classes)

#     def forward(self, x, adj, mask=None):
#         s = self.gnn1_pool(x, adj, mask)
#         x = self.gnn1_embed(x, adj, mask)

#         x, adj, l1, e1 = dense_diff_pool(x, adj, s, mask)

#         s = self.gnn2_pool(x, adj)
#         x = self.gnn2_embed(x, adj)

#         x, adj, l2, e2 = dense_diff_pool(x, adj, s)

#         x = self.gnn3_embed(x, adj)

#         x = x.mean(dim=1)
#         x = self.lin1(x).relu()
#         x = self.lin2(x)
#         return F.log_softmax(x, dim=-1), l1 + l2, e1 + e2

In [13]:
class mincutnet(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=16):
        super(mincutnet, self).__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels)
        num_of_centers =  20
        self.pool1 = Linear(hidden_channels, num_of_centers) # The degree of the node belonging to any of the centers
        self.conv2 = DenseGraphConv(hidden_channels, hidden_channels)

        #self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch): # x torch.Size([661, 3]),  data.batch  torch.Size([783])
        x = F.relu(self.conv1(x, edge_index))  #x torch.Size([661, 32])
        #print('x :',x.shape)
        x, mask = to_dense_batch(x, batch) #now x torch.Size([1, 661, 32]) ; mask torch.Size([20, 122])
        #print('mask :',mask.shape)
        
        adj = to_dense_adj(edge_index, batch) # adj torch.Size([1, 661, 661])
        s = self.pool1(x) # s torch.Size([1, 661, 20])
        #print('s :',s.shape)
        #print('adj :',adj.shape)
        
        x, adj, mincut_loss, ortho_loss = dense_mincut_pool(x, adj, s, mask) # x torch.Size([1, 20, 32]),  adj torch.Size([1, 20, 20])
        x = self.conv2(x, adj) #x torch.Size([1, 20, 32])
        g_emb = x.mean(dim=1) # x torch.Size([1, 32])
        #x = F.relu(self.lin1(x)) # x torch.Size([1, 32])
        out = self.lin2(g_emb) #x torch.Size([1, 2])
        return out, g_emb, mincut_loss, ortho_loss

In [ ]:
# class Net(torch.nn.Module):
#     def __init__(self, in_channels, out_channels, hidden_channels=32):
#         super(Net, self).__init__()
        
#         self.in_head = 2
#         self.out_head = 1
#         self.emb_dim = 16
        
#         max_nodes = 101
        

#         self.conv1 = GATConv(in_channels, hidden_channels, heads=self.in_head)
#         self.conv2 = GATConv(hidden_channels*self.in_head, hidden_channels,concat=False)
#         self.conv3 = GATConv(hidden_channels, self.emb_dim, concat=False, dropout=0.6)
        
#         num_nodes = ceil(0.20 * max_nodes)
#         self.pool1 = Linear(in_channels, num_nodes)
        
#         self.conv4 = GCNConv(self.emb_dim,  self.emb_dim)
        
#         self.lin = Linear(self.emb_dim, out_channels)

#     def forward(self, x, edge_index, batch): 
        
#         s = self.pool1(x)
        
#         print('s :',s.shape)

#         x = self.conv1(x, edge_index)
#         x = x.relu()
#         x = self.conv2(x, edge_index)
#         x = x.relu()
#         emb,attn_weights = self.conv3(x, edge_index,return_attention_weights=True)
        
#         print('emb :',emb.shape)
         
#         x, mask = to_dense_batch(emb, batch) 
#         adj = to_dense_adj(edge_index, batch)
        
#         print('adj :',adj.shape)
#         print('mask :',mask.shape)
        
         
#         x, adj,_ ,_ = dense_diff_pool(x, adj, s, mask) 
        
#         x = self.conv4(x, adj) 
#         g_emb = x.mean(dim=1)
        
#         out = self.lin(g_emb) #x torch.Size([1, 2])
#         return out, emb, g_emb, attn_weights
    
# model = Net(num_node_features, len(all_trials))
# print(model)

In [14]:
model = mincutnet(num_node_features, len(all_trials))
print(model)

mincutnet(
  (conv1): GCNConv(3, 16)
  (pool1): Linear(in_features=16, out_features=20, bias=True)
  (conv2): DenseGraphConv(16, 16)
  (lin2): Linear(in_features=16, out_features=10, bias=True)
)


In [15]:
#model = GCN(hidden_channels=32)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    
    for data in train_loader:# Iterate in batches over the training dataset.
        out, g_emb, _,_= model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        out, g_emb, _,_ =  model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

In [16]:
train_acc_lst = []
test_acc_lst = []
for epoch in range(1, 100):
    train()
    train_acc = test(train_loader)
    train_acc_lst.append(train_acc)
    test_acc = test(test_loader)
    test_acc_lst.append(test_acc)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.3780, Test Acc: 0.3560
Epoch: 002, Train Acc: 0.6172, Test Acc: 0.6280
Epoch: 003, Train Acc: 0.6972, Test Acc: 0.6820
Epoch: 004, Train Acc: 0.6436, Test Acc: 0.6420
Epoch: 005, Train Acc: 0.7992, Test Acc: 0.8020
Epoch: 006, Train Acc: 0.5660, Test Acc: 0.5680
Epoch: 007, Train Acc: 0.5592, Test Acc: 0.5700
Epoch: 008, Train Acc: 0.8080, Test Acc: 0.8140
Epoch: 009, Train Acc: 0.8264, Test Acc: 0.8100
Epoch: 010, Train Acc: 0.8156, Test Acc: 0.8000
Epoch: 011, Train Acc: 0.8528, Test Acc: 0.8560
Epoch: 012, Train Acc: 0.6996, Test Acc: 0.6960
Epoch: 013, Train Acc: 0.7372, Test Acc: 0.7420
Epoch: 014, Train Acc: 0.8488, Test Acc: 0.8200
Epoch: 015, Train Acc: 0.8448, Test Acc: 0.8500
Epoch: 016, Train Acc: 0.8688, Test Acc: 0.8780
Epoch: 017, Train Acc: 0.8456, Test Acc: 0.8340
Epoch: 018, Train Acc: 0.7836, Test Acc: 0.7780
Epoch: 019, Train Acc: 0.7536, Test Acc: 0.7680
Epoch: 020, Train Acc: 0.8028, Test Acc: 0.8040
Epoch: 021, Train Acc: 0.8448, Test Acc:

In [ ]:
# x : torch.Size([5613, 32])
# mask : torch.Size([64, 97])
# s : torch.Size([64, 97, 20])
# adj : torch.Size([64, 97, 97])

### Evaluate on a seperate set

In [17]:
# taking rest graph from each population as evaluation set

simu_eval_set = []
simu_eval_labels = []

for graph_set,graph_labels in zip(all_graphs_same_trial,all_graphs_labels_same_trial):
    simu_eval_set.extend(graph_set[300:])
    simu_eval_labels.extend(graph_labels[300:])

In [18]:
eval_set = []

for i,g in enumerate(simu_eval_set):
    graph_remove_dummy_nodes(g) # remove dummy nodes
    g.remove_edges_from(nx.selfloop_edges(g)) # remove self loop edges
    
    attr_coords = np.array(list(nx.get_node_attributes(g,'coord').values())) #simu attribute (coords) 
    x = torch.tensor(attr_coords,dtype=torch.float)

    #x = torch.tensor(nx.adjacency_matrix(g).todense(),dtype=torch.float)
    
    y = torch.tensor(simu_eval_labels[i],dtype=torch.long) # add graph label
    edge_index = torch.tensor(list(g.edges))
    
    eval_set.append(Data(x=x, y=y, edge_index=edge_index.t().contiguous()))

/tmp/ipykernel_7111/2408919390.py:12: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  y = torch.tensor(simu_eval_labels[i],dtype=torch.long) # add graph label


In [ ]:
reference_set = []
reference_label = list(np.arange(0,10,1))

for i,g in enumerate(referene_graphs):
    #graph_remove_dummy_nodes(g) 
    g.remove_edges_from(nx.selfloop_edges(g)) 
    
    attr_coords = np.array(list(nx.get_node_attributes(g,'coord').values())) #simu attribute (coords) 
    x = torch.tensor(attr_coords,dtype=torch.float)
    
    y = torch.tensor(reference_label[i],dtype=torch.long) # add graph label
    edge_index = torch.tensor(list(g.edges))
    
    reference_set.append(Data(x=x, y=y, edge_index=edge_index.t().contiguous()))

In [ ]:
# eval emb mixed with reference graph
all_graph = DataLoader(eval_set+reference_set, batch_size= len(eval_set) + len(reference_set), shuffle=False)

for data in all_graph:
    prob,g_emb,out_emb,attn_weights = model(data.x, data.edge_index, data.batch)
    
# out_emb = out_emb.detach().numpy()
# g_emb = g_emb.detach().numpy()

In [ ]:
def get_labels_from_k_means(k, coords):
    
    kmeans = KMeans(n_init= 'auto',n_clusters=k, random_state=0).fit(coords)
    
    return kmeans,kmeans.labels_

In [ ]:
k = 10      
kmeans,kmeans_labels = get_labels_from_k_means(k, g_emb[:2000])

In [ ]:
all_emb = np.concatenate((g_emb,kmeans.cluster_centers_),axis=0)

In [ ]:
from sklearn.manifold import TSNE
import pandas as pd

n_components = 2
tsne = TSNE(n_components)

tsne_graph = tsne.fit_transform(all_emb)
tsne_graph.shape

In [ ]:
sns.set(rc={'figure.figsize':(13,9)})

tsne_graph_df = pd.DataFrame({'tsne_1': tsne_graph[:2000,0], 'tsne_2': tsne_graph[:2000,1], 'label':simu_eval_labels})
tsne_ref_df = pd.DataFrame({'tsne_1': tsne_graph[2000:2010,0], 'tsne_2': tsne_graph[2000:2010,1], 'label':reference_label})
tsne_kmeans_df = pd.DataFrame({'tsne_1': tsne_graph[2010:,0], 'tsne_2': tsne_graph[2010:,1],'label':reference_label})

fig, ax = plt.subplots(1)
sns.scatterplot(x='tsne_1', y='tsne_2', hue='label', data=tsne_graph_df, ax=ax,s=20,palette='tab10')
sns.scatterplot(x='tsne_1', y='tsne_2', hue='label', data=tsne_ref_df, ax=ax,s=150,palette='tab10',markers='label')
plt.scatter(tsne_kmeans_df['tsne_1'], tsne_kmeans_df['tsne_2'], c='black', s = 100, marker='X')

# lim = (tsne_graph.min()-5, tsne_graph.max()+5)
# ax.set_xlim(lim)
# ax.set_ylim(lim)

ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

In [ ]:
from math import ceil

In [ ]:
ceil(0.25 * 80)

### Compare with classical methods on simulations

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(g_emb, simu_eval_labels, test_size=0.6, random_state=42)

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
clf_svm = SVC(gamma='auto',kernel='linear').fit(X_train, y_train)
clf_svm.score(X_test, y_test)

In [ ]:
clf_rf = RandomForestClassifier(max_depth = 4,random_state=0).fit(X_train, y_train)
clf_rf.score(X_test, y_test)

In [ ]:
# On coordinates

In [ ]:
avg_feature_coords = []

for d in eval_set:
    avg_feature_coords.append(np.mean(d.x.detach().numpy(),axis=0))
    
avg_feature_coords = np.array(avg_feature_coords)

In [ ]:
X_train_coords, X_test_coords, y_train_coords, y_test_coords = train_test_split(avg_feature_coords, simu_eval_labels, test_size=0.6, random_state=42)

In [ ]:
clf_coord = LogisticRegression(random_state=0).fit(X_train_coords, y_train_coords)
clf_coord.score(X_test_coords, y_test_coords)

In [ ]:
clf_svm_coord = SVC(gamma='auto',kernel='linear').fit(X_train_coords, y_train_coords)
clf_svm_coord.score(X_test_coords, y_test_coords)

In [ ]:
clf_rf_coord = RandomForestClassifier(max_depth = 5,random_state=0).fit(X_train_coords, y_train_coords)
clf_rf_coord.score(X_test_coords, y_test_coords)